In [1]:
import os
import sys
import time

import numpy as np

from pstools.rambo import dot

from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense, Activation
from tensorflow.keras import activations
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.optimizers import Adam
from tensorflow.keras import backend 

import matplotlib.pyplot as plt
%matplotlib inline

2025-11-17 08:59:53.805832: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Training a Neural Network to approximate amplitudes from pre-generated data ##

The sqaured scattering amplitude (summed and averaged over colour) for $e^+e^- \to$ jets have been evaluated over a flat phase-space generated with the RAMBO algorithm. Various phase-space cuts for the separation of final state 'jets' have been applied using the JADE algorithm. Tree/zero-loop '0L' ($|A^{(0)}|^2$) and one-loop ($2{\rm Re}(A^{(0)*}A^{(1)}|$) amplitudes have been compute using the NJet C++ library (https://bitbucket.org/njet/njet/wiki/Home)

The data consists of 1M phase-space points for processes with 3 or 4 final state jets. The data is given in numpy '.npz' format generated using the 'numpy.savez' command and are labelled as

<code>data/NJdata_\<loop order\>_ee\<n\>j_d\<jade cut\>.npz</code>

where loop order=0L,1L, n=3,4 and jade cut=0.03, 0.02 and 0.01

The data files can be obtained from the url (using wget for example):

`wget http://personalpages.to.infn.it/~badger/njet-data/eejet_data.tar.gz`

# Aims #

To use the tensorflow library to make a simple approximation of the multi-variable amplitude function and test how good of an approximation we get. Try to determine how well we approximate the amplitude point-by-point correlates with the observable cross section

# Step 1: Separate amplitude data and train TensorFlow neural network #

In [2]:
loop = 0
jets = 3
delta_cut = 0.03

In [3]:
NJdata = np.load("data/NJdata_"+str(loop)+"L_ee"+str(jets)+"j_d"+str(delta_cut)+".npz")
print(NJdata.files)

mom_data = NJdata['momenta']
amp_data = NJdata['NJ_vals']

['momenta', 'NJ_vals']


In [4]:
# choose the number of training points (will later be split into NN train/validation set)
n_points = len(amp_data)
print("found", n_points, "data points")

n_training_points = 100000
# choose the number of points for interpolation tests after training
# NB - different from the training/validation split during training
n_test_points = 100000

print("will train/validate on",n_training_points,"then test on",n_test_points)

found 1000000 data points
will train/validate on 100000 then test on 100000


In [5]:
# NB the points are already randomly distributed according to RAMBO
# so we may just slice the array into training/validation and testing
mom_train = mom_data[:n_training_points]
amp_train = amp_data[:n_training_points]

In [6]:
def mean_and_std(myarray, axis=0):
    return np.mean(myarray, axis=axis), np.std(myarray, axis=axis)

In [7]:
def standardize(myarray, ave, std):
    return (myarray-ave)/std

def destandardize(myarray, ave, std):
    return myarray*std+ave

In [8]:
# the standardized momenta cuts out the initial state since it is fixed for e+e- collisions
# NB if we attempted to keep it then dividing by the standard deviation would give division by zero
# momenta[:,2:,:] means take all phase space points but only take momenta 2 to n
# since the array is labelled momenta[i=(1,n_points), j=(1,n_particles), mu=(1,4)]  
mom_ave, mom_std = mean_and_std(mom_train[:,2:,:], axis=0)
mom_stdized = standardize(mom_train[:,2:,:], mom_ave, mom_std)

In [9]:
# alternatively we could use momentum invariants (p_i+p_j+...)^2 as inputs
# 3n-10 of these are independent for an n-particle scattering (masseless) amplitude
# 
def minkowski_dot_matrix(mm):
    g = np.array([1, -1, -1, -1])
    mm_tilde = mm * g
    A = np.einsum('ik,jk->ij', mm_tilde, mm)

    return A

def computeinvariants(mm):
    A = minkowski_dot_matrix(mm)
    inds = np.triu_indices_from(A, k=1)
    
    return A[inds]

inv_train = np.array([computeinvariants(mm) for mm in mom_train])

In [10]:
# as we are have e+e- with fixed centre-of-mass energy the invariant (p0+p1)^2 = 2*p0.p1 is constant and can be eliminated
inv_ave, inv_std = mean_and_std(inv_train[:,1:],axis=0)
inv_stdized = standardize(inv_train[:,1:], inv_ave, inv_std)

In [11]:
# the standardized the ampltide values as well
amp_ave = np.mean(amp_train)
amp_std = np.std(amp_train)
amp_stdized = (amp_train-amp_ave)/amp_std

In [12]:
# now set up Keras model with flattened momenta and/or invariants as input values
n_final = len(mom_stdized[0])
input_size_mom = n_final*4
input_values_mom = mom_stdized.reshape(-1,input_size_mom)

input_size_inv = len(inv_stdized[0])
input_values_inv = inv_stdized

In [13]:
backend.clear_session()

In [14]:
def make_baseline(layers, input_size):
    model = Sequential()
    model.add(Dense(layers[0], input_dim=(input_size)))
    model.add(Activation(activations.tanh))
        
    for layer in range(1, len(layers)):
        model.add(Dense(layers[layer]))
        model.add(Activation(activations.tanh))

    model.add(Dense(1))
    
    opt = Adam(learning_rate=1e-4, beta_1=0.9, beta_2=0.999, amsgrad=False)
    model.compile(optimizer=opt, loss='mean_squared_error', metrics=['mean_squared_error'])
    return model

callbacks = [
    EarlyStopping(monitor='val_loss', patience=50, min_delta=1e-4, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=1e-6, verbose=1)
]

In [15]:
input_values_mominv = np.concatenate([input_values_mom,input_values_inv], axis=1)

In [16]:
model_mom = make_baseline([20,40,20],len(input_values_mom[0]))
model_mom.summary()
model_inv = make_baseline([20,40,20],len(input_values_inv[0]))
model_mominv = make_baseline([20,40,20],len(input_values_mominv[0]))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 20)                260       
                                                                 
 activation (Activation)     (None, 20)                0         
                                                                 
 dense_1 (Dense)             (None, 40)                840       
                                                                 
 activation_1 (Activation)   (None, 40)                0         
                                                                 
 dense_2 (Dense)             (None, 20)                820       
                                                                 
 activation_2 (Activation)   (None, 20)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 2

2025-11-17 09:00:04.231214: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [17]:
in_mom_tr, in_mom_val, out_mom_tr, out_mom_val = train_test_split(input_values_mom, amp_stdized, test_size=0.2)
in_inv_tr, in_inv_val, out_inv_tr, out_inv_val = train_test_split(input_values_inv, amp_stdized, test_size=0.2)
in_mominv_tr, in_mominv_val, out_mominv_tr, out_mominv_val = train_test_split(input_values_mominv, amp_stdized, test_size=0.2)

In [ ]:
st = time.time();

history_mom = model_mom.fit(
    in_mom_tr, out_mom_tr,
    validation_data=(in_mom_val, out_mom_val),
    batch_size=1024, epochs=2000, verbose=0,
    callbacks=callbacks
)

et = time.time()
print('trained in ',(et-st)/60.,' mins')

print(f'Best case loss: {callbacks[0].best}')

In [ ]:
st = time.time();

history_inv = model_inv.fit(
    in_inv_tr, out_inv_tr,
    validation_data=(in_inv_val, out_inv_val),
    batch_size=1024, epochs=2000, verbose=0,
    callbacks=callbacks
)

et = time.time()
print('trained in ',(et-st)/60.,' mins')

print(f'Best case loss: {callbacks[0].best}')

In [ ]:
st = time.time();

history_mominv = model_mominv.fit(
    in_mominv_tr, out_mominv_tr,
    validation_data=(in_mominv_val, out_mominv_val),
    batch_size=1024, epochs=2000, verbose=0,
    callbacks=callbacks
)

et = time.time()
print('trained in ',(et-st)/60.,' mins')

print(f'Best case loss: {callbacks[0].best}')

In [ ]:
# Access the loss values from the training history.
train_loss_mom = history_mom.history['loss']
val_loss_mom = history_mom.history['val_loss']
train_loss_inv = history_inv.history['loss']
val_loss_inv = history_inv.history['val_loss']
train_loss_mominv = history_mominv.history['loss']
val_loss_mominv = history_mominv.history['val_loss']

# Create a plot of the training and validation loss over epochs.
epochs = range(1, len(train_loss_mom) + 1)
plt.plot(epochs, train_loss_mom, label='Training Loss (mom. inputs)')
plt.plot(epochs, val_loss_mom, label='Validation Loss (mom. inputs)')
epochs = range(1, len(train_loss_inv) + 1)
plt.plot(epochs, train_loss_inv, label='Training Loss (inv. inputs)')
plt.plot(epochs, val_loss_inv, label='Validation Loss (inv. inputs)')
epochs = range(1, len(train_loss_mominv) + 1)
plt.plot(epochs, train_loss_mominv, label='Training Loss (mom.+inv. inputs)')
plt.plot(epochs, val_loss_mominv, label='Validation Loss (mom.+inv. inputs)')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
#plt.ylim(0,0.01)
plt.legend()

# Step 2: Test the model on unseen data

In [ ]:
mom_test = mom_data[n_training_points:n_training_points+n_test_points]

In [ ]:
mom_test_stdized = standardize(mom_test[:,2:,:],mom_ave, mom_std).reshape(-1,input_size_mom)

inv_test = np.array([computeinvariants(mm) for mm in mom_test])
inv_test_stdized = standardize(inv_test[:,1:], inv_ave, inv_std)

mominv_test_stdized = np.concatenate([mom_test_stdized,inv_test_stdized], axis=1)

In [ ]:
amp_pred_mom = np.array(destandardize(
    model_mom.predict(mom_test_stdized),
    amp_ave, amp_std)).reshape(-1)

amp_pred_inv = np.array(destandardize(
    model_inv.predict(inv_test_stdized),
    amp_ave, amp_std)).reshape(-1)

amp_pred_mominv = np.array(destandardize(
    model_mominv.predict(mominv_test_stdized),
    amp_ave, amp_std)).reshape(-1)

In [ ]:
amp_test = amp_data[n_training_points:n_training_points+n_test_points]

In [ ]:
acc_mom = 2.*(amp_pred_mom-amp_test)/(amp_pred_mom+amp_test)
acc_inv = 2.*(amp_pred_inv-amp_test)/(amp_pred_inv+amp_test)
acc_mominv = 2.*(amp_pred_mominv-amp_test)/(amp_pred_mominv+amp_test)

In [ ]:
mybins = np.histogram_bin_edges(acc_mom, bins=100, range=(-3,3))
plt.hist(acc_mom, density=False, bins=mybins, alpha=0.5)
plt.hist(acc_inv, density=False, bins=mybins, alpha=0.5)
plt.hist(acc_mominv, density=False, bins=mybins, alpha=0.5)
plt.xlim([-3,3])
plt.xlabel('Accuracy = 2(Pred-Truth)/(Pred+Truth)|')
plt.ylabel('Frequency');

In [ ]:
logacc_mom = -np.log10(abs(acc_mominv))
logacc_inv = -np.log10(abs(acc_inv))
logacc_mominv = -np.log10(abs(acc_mominv))

mybins = np.histogram_bin_edges(logacc_mom, bins=100, range=(-1,5))
plt.hist(logacc_mom, density=False, bins=mybins, alpha=0.5, label='mom.')
plt.hist(logacc_inv, density=False, bins=mybins, alpha=0.5, label='inv.')
plt.hist(logacc_mominv, density=False, bins=mybins, alpha=0.5, label='mom.+inv.')
plt.xlim([-1,5])
plt.xlabel('Approx. Correct Digits = -Log10(|2(Pred-Truth)/(Pred+Truth)|)')
plt.ylabel('Frequency');
plt.legend()

# Step 3: Check the total cross-section #

Let's now check the cross section which for a RAMBO phase-space is defined as (Leading Order)

$\sigma = 1/N_{trials} \sum_p |A(p)|^2$

where $N_{trials}$ is total number of random samples of momenta generated. Since JADE cuts have been applied the number of trials will not be equal to the number of phase-space points. This information was not given with the data set so we won't get the normalisation of the cross-section correct but the convergence cross-section can be tested using the mean value

$\hat{\sigma} = \langle |A(p)|^2 \rangle_p$

with the error estimate given by the standard deviation divided by the square root of the number of points.

In [ ]:
# total cross section for the test sample
print("sigma_test = ",np.mean(amp_test),"+/-",np.std(amp_test)/np.sqrt(len(amp_test)))
sigma_ref = np.mean(amp_test)

In [ ]:
# total cross section using the three models
print("sigma_pred_mom = ",np.mean(amp_pred_mom),"+/-",np.std(amp_pred_mom)/np.sqrt(len(amp_pred_mom)))
print("sigma_pred_inv = ",np.mean(amp_pred_inv),"+/-",np.std(amp_pred_inv)/np.sqrt(len(amp_pred_inv)))
print("sigma_pred_mominv = ",np.mean(amp_pred_mominv),"+/-",np.std(amp_pred_mominv)/np.sqrt(len(amp_pred_mominv)))

In [ ]:
# Now plot the cross-section cumilatively for the test set and models 1,2
stepsize = 1000
xs_NJ  = [[np.mean(amp_test[0:stepsize*i]), np.std(amp_test[0:stepsize*i])] for i in range(1,int(n_test_points/stepsize))]
xs_NN1 = [[np.mean(amp_pred_mom[0:stepsize*i]), np.std(amp_pred_mom[0:stepsize*i])] for i in range(1,int(n_test_points/stepsize))]
xs_NN2 = [[np.mean(amp_pred_inv[0:stepsize*i]), np.std(amp_pred_inv[0:stepsize*i])] for i in range(1,int(n_test_points/stepsize))]
xs_NN3 = [[np.mean(amp_pred_mominv[0:stepsize*i]), np.std(amp_pred_mominv[0:stepsize*i])] for i in range(1,int(n_test_points/stepsize))]

xs_NJ = np.array(xs_NJ)
xs_NN1 = np.array(xs_NN1)
xs_NN2 = np.array(xs_NN2)
xs_NN3 = np.array(xs_NN3)

In [ ]:
# This notation takes every 100 entries in the cumalative less so we don't plot so many points
plotdata1 = xs_NJ[:,0]
plotdata2 = xs_NN1[:,0]
plotdata3 = xs_NN2[:,0]
plotdata4 = xs_NN3[:,0]

In [ ]:
# check that we did the cumalative average correctly
np.mean(amp_test)-plotdata1[-1]

In [ ]:
plt.plot(stepsize*np.array(range(len(plotdata1))), plotdata1, 'b-', lw=5, alpha=0.5, label='NJet')
plt.plot(stepsize*np.array(range(len(plotdata2))), plotdata2, 'r-', label='mom.')
plt.plot(stepsize*np.array(range(len(plotdata3))), plotdata3, 'g-', label='inv.')
plt.plot(stepsize*np.array(range(len(plotdata4))), plotdata4, 'y-', label='mom.+inv.')
plt.xlim([0,n_test_points])
plt.ylim([sigma_ref*(1.-0.1),sigma_ref*(1.+0.1)])
plt.ylabel('sigma')
plt.xlabel('iteration');
plt.title(r'e+e- -> '+str(jets)+'j at '+str(loop)+'-loop with JADE cut '+str(delta_cut))
plt.legend()

We may also look at the size of the amplitude as a function of the accuracy of the predicitions

In [ ]:
idx = np.random.choice(len(amp_test), size=5000, replace=False)

plt.scatter(logacc_mom[idx],np.log10(np.abs(amp_test[idx])),color='blue',alpha=0.7, label='mom.')
plt.scatter(logacc_inv[idx],np.log10(np.abs(amp_test[idx])),color='orange',alpha=0.7, label='inv.')
plt.scatter(logacc_mominv[idx],np.log10(np.abs(amp_test[idx])),color='lightgreen',alpha=0.7, label='mom.+inv.')
plt.xlabel('Approx. Correct Digits')
plt.ylabel('log10(|amplitude|)');
plt.legend()

# Exercises

1. Keeping all the parameters unchanged, re-run the whole notebook. Does it makes sense that we don't get the same answer?
2. Change the number of final state jets and the JADE cut 'delta_cut', does changing the network architecture improve the fit?
3. Choose one-loop e+e- -> 3j and check the performance. Does it match your expectations?
4. Does the activation function have an effect